### Import everthing

In [1]:
#make sure your drive path is correct! 
# macOS/OS X
# drive_path = '/Volumes/Brain2017/data/dynamic-brain-workshop/brain_observatory_cache/'

# Linux (will vary; the following is possibly what Ubuntu will do)
# drive_path = '/media/Brain2017/data/dynamic-brain-workshop/brain_observatory_cache'

# AWS
drive_path = '/data/dynamic-brain-workshop/brain_observatory_cache/'

# We need to import these modules to get started
import numpy as np
import pandas as pd
import os
import sys
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

from allensdk.core.brain_observatory_cache import BrainObservatoryCache

manifest_file = os.path.join(drive_path,'brain_observatory_manifest.json')
print manifest_file

boc = BrainObservatoryCache(manifest_file=manifest_file)

/data/dynamic-brain-workshop/brain_observatory_cache/brain_observatory_manifest.json


In [2]:
from allensdk.brain_observatory.natural_scenes import NaturalScenes
from allensdk.brain_observatory.static_gratings import StaticGratings
from allensdk.brain_observatory.natural_movie import NaturalMovie
from scipy.stats import binned_statistic
from sklearn.decomposition import PCA
from sklearn.manifold import SpectralEmbedding, MDS
from sklearn.covariance import EmpiricalCovariance
from scipy.spatial.distance import squareform, pdist

scriptpath = '/home/arib/SWDB_2017/swdb_2017_tools'
sys.path.append(scriptpath)

from swdb2017.brain_observatory.RSA_with_options import *

from swdb2017.brain_observatory.get_rsm import *

### Get all experiement ids

In [3]:
# Download a list of all targeted areas
targeted_structures = boc.get_all_targeted_structures()
print 'all targeted structures: ' + str(targeted_structures)

# Download a list of all imaging depths
depths = boc.get_all_imaging_depths()
print 'all imaging depths: ' + str(depths)

# Download a list of all cre driver lines 
cre_lines = boc.get_all_cre_lines()
print 'all cre lines: ' + str(cre_lines)

all targeted structures: [u'VISal', u'VISam', u'VISl', u'VISp', u'VISpm', u'VISrl']
all imaging depths: [175, 265, 275, 300, 320, 325, 335, 350, 365, 375, 435]
all cre lines: [u'Cux2-CreERT2', u'Emx1-IRES-Cre', u'Nr5a1-Cre', u'Rbp4-Cre_KL100', u'Rorb-IRES2-Cre', u'Scnn1a-Tg3-Cre']


In [4]:
all_exps_df = pd.DataFrame(boc.get_experiment_containers(targeted_structures= targeted_structures,
                                                     cre_lines= cre_lines, imaging_depths = depths))

rsa_types = ['rsa_mean_mean_mahala','rsa_random_mean_mahala', 'rsa_mean_mean_spectral_embedding', 'rsa_mean_mean_PCA']
for rsa_ in rsa_types:
    all_exps_df[rsa_] = 0
    all_exps_df[rsa_] = all_exps_df[rsa_].astype(object)


all_exps_df.head()

,cre_line,donor_name,failed,id,imaging_depth,reporter_line,specimen_name,tags,targeted_structure,rsa_mean_mean_mahala,rsa_random_mean_mahala,rsa_mean_mean_spectral_embedding,rsa_mean_mean_PCA
0,Cux2-CreERT2,283284,False,566759225,275,Ai93(TITL-GCaMP6f),Cux2-CreERT2;Camk2a-tTA;Ai93-283284,[],VISam,0,0,0,0
1,Nr5a1-Cre,261969,False,546328009,350,Ai93(TITL-GCaMP6f),Nr5a1-Cre;Camk2a-tTA;Ai93-261969,[],VISal,0,0,0,0
2,Scnn1a-Tg3-Cre,230570,False,511510911,350,Ai93(TITL-GCaMP6f),Scnn1a-Tg3-Cre;Camk2a-tTA;Ai93-230570,[],VISp,0,0,0,0
3,Emx1-IRES-Cre,284669,False,569792815,375,Ai93(TITL-GCaMP6f),Emx1-IRES-Cre;Camk2a-tTA;Ai93-284669,[],VISam,0,0,0,0
4,Rbp4-Cre_KL100,234584,False,511511015,375,Ai93(TITL-GCaMP6f),Rbp4-Cre;Camk2a-tTA;Ai93-234584,[],VISpm,0,0,0,0


In [5]:
exp_id = all_exps_df.loc[0,'id']

##### Run for all natural scene sessions

### Full pipeline

In [6]:
def get_all_data(exp_id, only_responsive = True, stimuli = ['natural_scenes', 'static_gratings', 'natural_movie_one']):
    """Gets all the data. We'll only use session B.
    
    Returns a matrix of trials x n_neurons, and another of frames x neurons.
    
    Eye position
    """
    dfs = []

    for stim in stimuli:
        session = boc.get_ophys_experiments(experiment_container_ids=[exp_id], stimuli=[stim])
        
        session_id =[session[i]['id'] for i in range(len(session)) 
                         if session[i]['session_type']=='three_session_B'][0]
        if session_id==[]: 
            # This stimuli does not exist.
            print('No '+stim+' for this experiment: ' + str(exp_id))
            continue
            
        data_set = boc.get_ophys_experiment_data(ophys_experiment_id=session_id)
        analysis_path = os.path.join(drive_path, 'ophys_experiment_analysis')
        analysis_file = os.path.join(analysis_path, str(session_id) + '_three_session_B_analysis.h5')
        
        if stim == 'natural_scenes':  
            ns = NaturalScenes.from_analysis_file(data_set, analysis_file)
            df = ns.mean_sweep_response
            
#             avg_locs, std_locs = get_mean_pupil(data_set, stim)
#             df['mean_pupil_loc_x'] = avg_locs[:,0]
#             df['mean_pupil_loc_y'] = avg_locs[:,1]
#             df['pupil_loc_std_x'] = std_locs[:,0]
#             df['pupil_loc_std_y'] = std_locs[:,1]
            
            df['frame'] = data_set.get_stimulus_table('natural_scenes').frame
            df.columns = df.columns.astype(str)
            df['type'] = stim

            dfs.append(df)
            
            

        elif stim == 'static_gratings':
            sg = StaticGratings.from_analysis_file(data_set, analysis_file)
            df = sg.mean_sweep_response
            
#             avg_locs, std_locs = get_mean_pupil(data_set, stim)
#             df['mean_pupil_loc_x'] = avg_locs[:,0]
#             df['mean_pupil_loc_y'] = avg_locs[:,1]
#             df['pupil_loc_std_x'] = std_locs[:,0]
#             df['pupil_loc_std_y'] = std_locs[:,1]

            stim_table = data_set.get_stimulus_table('static_gratings')
            df = pd.merge(df,stim_table,left_index=True, right_index=True)
            df.columns = df.columns.astype(str)
            df['type'] = stim
            dfs.append(df)

        elif stim =='natural_movie_one':
            nm = NaturalMovie.from_analysis_file(data_set, analysis_file, 'natural_movie_one')
            df = pd.DataFrame(bin_movie_response(nm))
            
#             avg_locs, std_locs = get_mean_pupil(data_set, stim)
#             df['mean_pupil_loc_x'] = avg_locs[:,0]
#             df['mean_pupil_loc_y'] = avg_locs[:,1]
#             df['pupil_loc_std_x'] = std_locs[:,0]
#             df['pupil_loc_std_y'] = std_locs[:,1]

            stim_table = data_set.get_stimulus_table('natural_movie_one')
            df = pd.merge(df,stim_table[['repeat','frame']],left_index=True, right_index=True)
            df.columns = df.columns.astype(str)
            df['type'] = stim

            dfs.append(df)
            
            
        else:
            raise Exception('stimulus not found')


        # drop unresponsive cells
        
            
    response = pd.concat(dfs)
    
    return response, ns.numbercells

In [7]:
def get_mean_pupil(data_set, stimuli):
    stim_table = data_set.get_stimulus_table(stimuli)
    avg_locs = []
    std_locs = []
    pup_loc = data_set.get_pupil_location()
    if stimuli == 'natural_movie_one':
        for b in range(150*10):
            start = stim_table.loc[b*6,'start']
            end = stim_table.loc[b*6+5,'end']
            these_pup_locs = pup_loc[1][start:end]
            avg_locs.append(np.nanmean(these_pup_locs, axis=0))
            std_locs.append(np.nanstd(these_pup_locs, axis=0))
    else:        
        for s in range(len(stim_table)):
        # these are frame presentation numbers
            start = stim_table.loc[s,'start'].astype(int)
            end = stim_table.loc[s,'end'].astype(int)

            these_pup_locs = pup_loc[1][start:end]
            avg_locs.append(np.nanmean(these_pup_locs, axis=0))
            std_locs.append(np.nanstd(these_pup_locs, axis=0))
    return np.array(avg_locs), np.array(std_locs)

In [8]:
def bin_movie_response(nm):
    sw = nm.get_sweep_response()
    # first let's build a matric of n_scans, n_images
    trial_responses = []
    for repeat in range(10):
        resp = sw.iloc[repeat]
        nneurons = resp.shape[0]
        n_frames = resp[0].shape[0]

        # we'll use 200 ms bins, a.k.a. 6 frames
        bin_size = 6
        this_response = np.zeros((n_frames//bin_size ,nneurons))
        for n in range(nneurons):
            this_response[:,n] = binned_statistic(np.arange(n_frames), resp[n], statistic='mean',
                                     bins =n_frames//bin_size )[0]
        trial_responses.append(this_response)   
            
    response = np.vstack(trial_responses)
        
    return response


In [9]:
def get_mean_responses(response, method = 'mean'):
    """method = 'mean' or 'random"""
    gratings = response.groupby('type').get_group('static_gratings')
    movies = response.groupby('type').get_group('natural_movie_one')
    scenes = response.groupby('type').get_group('natural_scenes')


    if method =='random':
        grating_response = gratings.groupby(('orientation', 'phase','spatial_frequency')).apply(
                                    lambda x : x.iloc[np.random.randint(0,len(x))])
        movie_response = movies.groupby('repeat').apply(
                                    lambda x : x.iloc[np.random.randint(0,len(x))])
        scene_response = scenes.groupby('frame').apply(
                                    lambda x : x.iloc[np.random.randint(0,len(x))])
    elif method =='mean':
        grating_response = gratings.groupby(('orientation', 'phase','spatial_frequency')).apply(
                                    lambda x : x.mean())
        movie_response = movies.groupby('repeat').apply(
                                    lambda x : x.mean())
        scene_response = scenes.groupby('frame').apply(
                                    lambda x : x.mean())

    mean_response = pd.concat([grating_response, movie_response, scene_response])
    return mean_response

In [10]:
def get_precision(all_responses):
        #using sklearn for its nice approximate inverse methods
    if response.shape[0] < response.shape[1]:
        raise('Must have more stimuli responses than neurons')

    emp_cov = EmpiricalCovariance()
    emp_cov.fit(response)
    precision = emp_cov.get_precision()
    
    return precision

In [ ]:

for i, r in all_exps_df.iterrows():
    if i<1:continue
    print i
    exp_id = all_exps_df['id'][i]
    response_df, numbercells = get_all_data(exp_id)
    response = response_df.iloc[:,:numbercells].values
    
    mean_response = get_mean_responses(response_df, method = 'mean')
    mean_response = mean_response.iloc[:,:numbercells].values
    
    precision = get_precision(response)

    
    ### Get RSAs

   
    rsa_mean_mean_mahala = get_representational_similarity_response(mean_response, corr = 'mahalanobis', 
                                                                    precision = precision)
    all_exps_df.set_value(i,'rsa_mean_mean_mahala', rsa_mean_mean_mahala)

    rsa_random_mean_mahala = get_representational_similarity_response(mean_response, corr = 'mahalanobis', 
                                                                    precision = precision)
    all_exps_df.set_value(i,'rsa_random_mean_mahala', rsa_random_mean_mahala)
    
#     se = SpectralEmbedding(n_components=50, n_jobs = -1)
#     se.fit(response)
#     rsa_mean_mean_SE = get_representational_similarity_response(mean_response, corr = 'euclidean', 
#                                                                    prefit_embedder = se)
#     all_exps_df.set_value(i,'rsa_mean_mean_spectral_embedding', rsa_mean_mean_SE)
    
    
    pca = PCA(whiten = True, n_components=min(50,numbercells-1))
    pca.fit(response)
    rsa_mean_mean_PCA = get_representational_similarity_response(mean_response, corr = 'euclidean', 
                                                                    prefit_embedder = pca)
    all_exps_df.set_value(i,'rsa_mean_mean_PCA', rsa_mean_mean_PCA)
    
    all_exps_df.to_hdf('exps_w_RSA.h5','all_exps_df')



1
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/546698458_three_session_B_analysis.h5 in read-only mo

Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/501889084_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/501889084_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/501889084_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/501889084_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/501889084_three_session_B_analysis.h5 in read-only mode
Opening /data/dynamic-brain-workshop/brain_observatory_cache/ophys_experiment_analysis/501889084_three_session_B_analysis.h5 in read-only mode


In [ ]:
all_exps_df.to_hdf('exps_w_RSA.h5','all_exps_df')

In [ ]:
for i, r in all_exps_df.iterrows():
    print i
    exp_id = all_exps_df['id'][i]
    response_df, numbercells = get_all_data(exp_id)
    response = response_df.iloc[:,:numbercells].values
    
    mean_response = get_mean_responses(response_df, method = 'mean')
    mean_response = mean_response.iloc[:,:numbercells].values
    

    
    ### Get RSAs


    
    se = SpectralEmbedding(n_components=min(50,numbercells-1), n_jobs = -1)
    se.fit(response)
    rsa_mean_mean_SE = get_representational_similarity_response(mean_response, corr = 'euclidean', 
                                                                   prefit_embedder = se)
    all_exps_df.set_value(i,'rsa_mean_mean_spectral_embedding', rsa_mean_mean_SE)

    
    all_exps_df.to_hdf('exps_w_RSA.h5','all_exps_df')

In [12]:
numbercells

18